# Resource Augmented Generation (RAG)


## Overview

- Motivation for RAG
- Idea behind RAG
- Advantages and Disadvantages
- Implementation to augment question + answer
- Advanced applications


#### Imagine you went to live under a rock on August 2006. When you come out in 2024, you are asked how many planets revolve around the sun. What would you say?...

![pluto](https://github.com/nieve-consulting/genai-learning-sprint/blob/main/labs/03-rag/images/pluto_planets.jpeg?raw=1)

This is similar to LLMs which are trained with data until a certain point and then asked questions on data they are not trained on. Understandably, LLMs will either be unable to answer or simply hallucinate a probably wrong answer.


### What can be done?

Have the LLM go to the library using **Research Augmented Generation (RAG)**!


RAG involves adding your own data (via a retrieval tool) to the prompt that you pass into a large language model.


![rag architecture](https://github.com/nieve-consulting/genai-learning-sprint/blob/main/labs/03-rag/images/rag-overview.original.png?raw=1)

Image credit: https://scriv.ai/guides/retrieval-augmented-generation-overview/


RAG has been shown to improve LLM prediction accuracy without needing to increase parameter size.

![rag architecture](https://github.com/nieve-consulting/genai-learning-sprint/blob/main/labs/03-rag/images/rag_acc_v_size.png?raw=1)

_Image credit: Yu, Wenhao. "Retrieval-augmented generation across heterogeneous knowledge." Proceedings of the 2022 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies: Student Research Workshop. 2022._


RAG also increases explainability by giving the source for information.

![rag architecture](https://github.com/nieve-consulting/genai-learning-sprint/blob/main/labs/03-rag/images/rag_source_locator.png?raw=1)

Image credit: https://ai.stanford.edu/blog/retrieval-based-NLP/


## Advantages and Disadvantages

### Advantages

- Provides domain specific context
- Improves predictive performance and reduces hallucinations
- Does not increase model parameters
- Less labor intensive than fine-tuning LLMs

### Disadvantages

- May introduce latency since we are adding a relatively costly search step
- If your dataset includes private information, you may inadvertently expose another user with this information.
- The data you want to use needs to be curated and you should decide how the data should be accessed. This adds time for the initial set-up.


# Implementation


### 1. Install + load relevant modules:

- langchain
- torch
- transformers
- sentence-transformers
- datasets
- faiss-cpu
- pypdf
- unstructure[pdf]
- huggingface_hub (add hf_token)


In [ ]:
!apt install poppler-utils tesseract-ocr -y

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install torch
!pip install transformers
!pip install faiss-cpu
!pip install pypdf
!pip install sentence-transformers
!pip install unstructured
!pip install unstructured[pdf]
!pip install tiktoken
!pip install huggingface_hub
!pip install --upgrade nltk
!pip install --upgrade bitsandbytes
!pip install einops
!pip install xformers
!pip install chromadb
!pip install accelerate

### 2. Choose a dataset to use and then load it into your code

Here we are using the pdfs loaded in pdfs/. We load this using langchain DirectoryLoader.


We can load multiple types of datasets into this example though the most commonly used are PDFs and websites.

To load websites, we could also use `langchain WebBaseLoader`

In this example, we will consider PDFs and load them in using `langchain DirectoryLoader`.


In [ ]:
import nltk

nltk.download("punkt_tab")

In [ ]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader("./PDFs", glob="**/*.pdf", show_progress=True)
documents = loader.load()

### 3. Now, we need to split our documents into chunks.

We want the embedding to be greater than 1 word but much less than an entire page. This is essential for the similarity search between the query and the document. Essentially, the query will be searched for greatest similarity to embedded chunks in the dataset. Then those chunks with greatest similarity are augmented to the query.

It is essential to choose the chunking method according to your data type.
There are different ways to do this:

Fixed size

- Token: Splits text on tokens. Can chunk tokens together
- Character: Splits based on some user defined character.

Recursive

- Recursively splits text. Useful for keeping related pieces of text next to each other.

Document based

- HTML: Splits text based on HTML-specific characters.
- Markdown: Splits on Markdown-specific characters
- Code: Splits text based on characters specific to coding languages.

Semantic chunking

- Extract semantic meaning from embeddings and then assess the semantic relationship between these chunks. Essentially splits into sentences, then groups into groups of 3 sentences, and then merges one that are similar in the embedding space.

Here we use recursive where the dataset is split using a set of characters. The default characters provided to it are ["\n\n", "\n", " ", ""]. A large text is split by the first character \n\n. If the first split by \n\n is still large then it moves to the next character which is \n and tries to split by it. This continues until the chunk size is reached.


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(documents)

### 4. Then we embed the chunked texts using a Transformer.

This allows us to encode the text into our search


Embedding converts text to a numerical representation in a vector space. RAG compares the embeddings of user queries within the vector of the knowledge library.

In this example, we choose a simple embedding using the MiniLM


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

modelPath = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

### 5.Create a vector database

Vector databases, also called vector storage, efficiently store and retrieve vector data, which are arrays of numerical values representing points in multi-dimensional space. They're useful for handling data like embeddings from deep learning models or numerical features. Unlike traditional relational databases, which aren't optimized for vectors, vector databases offer efficient storage, indexing, and querying for high-dimensional and variable-length vectors.

There are various types of vector databases:

1. Chroma
2. FAISS
3. Pinecone
4. Weaviate
5. Qdrant

Here, we build this using the FAISS utility.


![vector_database](https://github.com/nieve-consulting/genai-learning-sprint/blob/main/labs/03-rag/images/vector_database.png?raw=1)

Image credit: https://blog.gopenai.com/primer-on-vector-databases-and-retrieval-augmented-generation-rag-using-langchain-pinecone-37a27fb10546


In [ ]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)
question = "What is RF Fold?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

### 6. Initialize the LLM that will be used for question answering


Here, we use a pretrained model flan-t5-large as part of a HuggingFacePipeline. This will later be chained with the vector database for RAG.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain import HuggingFacePipeline

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={"temperature": 0, "max_length": 2048},
)
#'HuggingFaceH4/zephyr-7b-beta'

### 7. Retrieve data and use it to answer a question


![rag_workflow](https://github.com/nieve-consulting/genai-learning-sprint/blob/main/labs/03-rag/images/rag_workflow.png?raw=1)

Image credit: https://blog.gopenai.com/retrieval-augmented-generation-101-de05e5dc21ef


Let's ask questions it would only be able to know if the model actually read the texts!


In [9]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)
result = qa_chain(
    {"query": "What technique proposed in 2023 can be used to predict protein folding?"}
)
print(result["result"])

Now let's ask the chain where to find the article related to RFDiffusion


In [ ]:
qa_chain(
    {
        "query": "Which scientific article should I read to learn about RFdiffusion for protein folding?"
    }
)

## Exercise

Use any of the frameworks/models here to load in your favorite websites and ask the model a question regarding them.

Hint:

```
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(["https://www.espn.com/", "https://www.google.com"])

To bypass SSL verification errors during fetching, you can set the “verify” option:

loader.requests_kwargs = {‘verify’:False}

data = loader.load()
```


## More applications


### RAG using Llama 2, Langchain and ChromaDB


Load in models and setup pipeline


In [13]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time

# import chromadb
# from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

model_id = "meta-llama/Llama-2-7b-chat-hf"

device = f"cuda"

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16,
)

In [ ]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

In [ ]:
time_1 = time()
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare function to test pipeline


In [16]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,
    )
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [ ]:
test_model(
    tokenizer,
    query_pipeline,
    "Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.",
)

Set up Huggingface pipeline


In [ ]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
llm(
    prompt="Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words."
)

Load in data


In [19]:
loader = TextLoader("./state_union2023.txt", encoding="utf8")
documents = loader.load()

Chunk data recursively


In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [ ]:
all_splits

Embed and store in Chroma Vector store


In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cpu"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

In [23]:
vectordb = Chroma.from_documents(
    documents=all_splits, embedding=embeddings, persist_directory="chroma_db"
)

Set up chain


In [24]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, verbose=True
)

Test RAG


In [25]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

Get sources...


In [ ]:
query = "Which scientific article should I read to learn about RFdiffusion for protein folding?"
docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()["kwargs"]
    print("Source: ", doc_details["metadata"]["source"])
    print("Text: ", doc_details["page_content"], "\n")

In [ ]:
query = "What were the main topics in the State of the Union in 2023? Summarize. Keep it under 200 words."
test_rag(qa, query)